# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np

# Instance Field

In [2]:
# hyperparameters
block_size = 8
batch_size = 32 # how many independent sequences will be processed in parallel?
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_interval = 300
eval_iters = 200
learning_rage = 1e-2
max_iters = 3000
n_embd = 32

# Main

In [3]:
with open('../data/shakespeare.txt', 'r', encoding='utf-8') as f:
    shakespeare = f.read()
    
shakespeare[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [4]:
chars = sorted(list(set(shakespeare)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [6]:
data = torch.tensor(encode(shakespeare), dtype=torch.long)
data[:10]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [7]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [10]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
# print(xb)
print('targets:')
print(yb.shape)
# print(yb) 

print('----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size):
#         context = xb[b, :t+1]
#         target = yb[b, t]
#         print(f"When input is {context.tolist()} the target: {target}")

inputs:
torch.Size([32, 8])
targets:
torch.Size([32, 8])
----


In [11]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
#         self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
#         pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb
#         x = tok_emb + pos_emb #(B, T, C)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
        
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    #---------------------------------------------------------
    
    
    # GENERATOR
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # sample from distribution
            print(f"probs = {probs}")
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long).to(device), max_new_tokens=100)[0].tolist()))

tensor(4.3284, device='cuda:0', grad_fn=<NllLossBackward0>)


In [12]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [13]:
for iter in range(max_iters):
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    #sample a batch of data
    xb, yb = get_batch("train")
    
    #evealuate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3227, val loss 4.3099
step 300: train loss 2.9002, val loss 2.9061
step 600: train loss 2.6409, val loss 2.6726
step 900: train loss 2.5640, val loss 2.5863
step 1200: train loss 2.5426, val loss 2.5636
step 1500: train loss 2.5114, val loss 2.5383
step 1800: train loss 2.5063, val loss 2.5297
step 2100: train loss 2.5020, val loss 2.5138
step 2400: train loss 2.4920, val loss 2.5138
step 2700: train loss 2.4989, val loss 2.5118


In [14]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long).to(device), max_new_tokens=500)[0].tolist()))

probs = tensor([[1.5011e-01, 3.4180e-03, 1.3903e-04, 9.7094e-05, 7.4592e-05, 8.0379e-03,
         6.6785e-04, 3.3588e-04, 2.2299e-04, 6.1828e-04, 8.5939e-04, 2.5863e-04,
         2.4582e-04, 1.0300e-01, 3.2549e-02, 3.0663e-02, 1.7430e-02, 9.2754e-03,
         3.6288e-02, 2.0793e-02, 4.8211e-02, 6.7655e-02, 4.8415e-03, 1.1037e-02,
         2.6119e-02, 4.3469e-02, 2.5727e-02, 2.7122e-02, 2.4255e-02, 3.8896e-03,
         1.3151e-02, 5.3935e-02, 1.0803e-01, 6.5271e-03, 3.3065e-03, 5.5698e-02,
         2.4190e-04, 1.6826e-02, 2.4057e-04, 3.1404e-03, 1.7738e-03, 1.4601e-03,
         1.5329e-03, 1.1505e-03, 3.8616e-03, 8.1815e-04, 2.9100e-03, 1.0836e-03,
         3.5968e-04, 2.0703e-04, 2.3459e-03, 1.0042e-03, 3.1438e-03, 1.0391e-03,
         1.4751e-03, 2.2191e-04, 8.9882e-04, 6.2996e-03, 5.6036e-03, 1.1922e-03,
         1.3076e-04, 1.6837e-03, 8.3197e-05, 1.1012e-03, 1.0888e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
probs = tensor([[1.5011e-01, 3.4180e-03, 1.3903e-04, 9.7094

probs = tensor([[4.7044e-03, 1.3409e-01, 1.2008e-03, 1.6518e-05, 1.1868e-05, 4.7302e-03,
         1.1850e-02, 5.0929e-04, 3.1166e-03, 8.0318e-06, 2.8832e-03, 1.1676e-03,
         1.1132e-03, 3.6956e-05, 8.1836e-05, 8.5176e-05, 8.5617e-05, 3.2630e-05,
         1.1368e-04, 7.1891e-05, 8.3210e-05, 1.0759e-04, 1.8476e-05, 9.1854e-05,
         1.1829e-04, 9.0237e-05, 4.9302e-05, 1.5656e-05, 6.6214e-05, 1.1844e-05,
         1.2494e-04, 5.3916e-05, 5.8824e-05, 3.9376e-05, 3.1001e-05, 5.3758e-05,
         7.8774e-06, 2.2356e-05, 1.1370e-05, 4.9997e-03, 1.1747e-02, 3.8002e-02,
         8.3306e-03, 3.8799e-02, 2.4712e-03, 3.2057e-02, 1.8435e-03, 1.5275e-02,
         2.4377e-04, 8.4089e-03, 8.4818e-02, 1.6969e-02, 7.7254e-02, 4.3483e-03,
         3.9498e-02, 1.5080e-04, 1.8075e-01, 1.4575e-01, 1.0954e-01, 3.0964e-03,
         2.3682e-03, 1.4729e-03, 2.7977e-04, 4.4144e-03, 1.5716e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
probs = tensor([[1.2726e-02, 6.5088e-02, 1.3869e-03, 6.6493

probs = tensor([[2.1790e-03, 4.6907e-02, 6.2583e-04, 5.5748e-05, 2.7336e-05, 2.5457e-03,
         2.9257e-03, 8.0372e-04, 1.1332e-03, 3.3861e-05, 3.3048e-04, 5.5199e-04,
         6.3465e-04, 1.3259e-05, 9.8713e-05, 1.3282e-04, 1.9213e-04, 4.7161e-05,
         1.5395e-04, 7.4976e-05, 4.6023e-05, 3.0103e-05, 5.3887e-05, 1.5699e-04,
         1.8817e-04, 3.2155e-04, 2.1320e-04, 2.4048e-05, 1.2361e-04, 3.7065e-05,
         2.8848e-04, 2.3706e-04, 1.1532e-04, 2.3082e-05, 5.1286e-05, 1.5624e-04,
         2.3232e-05, 8.6064e-05, 1.9325e-05, 1.2830e-03, 1.1137e-02, 3.1788e-02,
         4.1026e-02, 3.8240e-03, 8.5768e-03, 1.5101e-02, 1.2520e-03, 3.7456e-02,
         1.2221e-03, 3.2392e-02, 8.3358e-02, 3.4541e-02, 1.7033e-01, 8.9886e-04,
         1.7063e-02, 1.3443e-04, 1.1984e-01, 6.9029e-02, 1.4341e-01, 1.0979e-02,
         4.8150e-02, 1.0155e-02, 1.4242e-04, 4.4418e-02, 8.2392e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
probs = tensor([[1.7892e-04, 2.8211e-03, 1.7159e-04, 5.1784

probs = tensor([[2.4355e-01, 7.4016e-01, 5.5297e-05, 1.4581e-06, 8.8490e-07, 1.9870e-03,
         7.1980e-04, 2.2717e-03, 1.6601e-04, 1.5150e-06, 6.8786e-04, 4.8036e-05,
         5.5479e-05, 2.9302e-05, 4.0714e-05, 3.4743e-05, 1.7654e-05, 5.7323e-06,
         8.8673e-06, 2.2653e-05, 2.3491e-05, 4.8606e-05, 9.9154e-07, 1.8804e-05,
         7.0931e-05, 1.3045e-05, 1.0344e-05, 1.0677e-05, 5.5670e-06, 2.0988e-06,
         4.3150e-05, 1.5743e-05, 3.5063e-05, 1.8983e-04, 1.2357e-05, 1.8506e-05,
         1.6146e-06, 5.5036e-06, 2.9039e-06, 1.1840e-04, 9.7350e-05, 6.6986e-05,
         1.2517e-03, 4.2992e-04, 3.7822e-04, 1.5468e-04, 8.4079e-06, 1.8262e-04,
         5.1278e-06, 6.1110e-05, 5.2353e-04, 3.4701e-04, 1.0548e-03, 3.7952e-04,
         2.4524e-04, 4.1938e-06, 8.4515e-04, 2.0895e-03, 7.0170e-04, 2.1849e-04,
         1.6910e-04, 1.7547e-04, 3.1683e-06, 8.3138e-05, 2.4652e-06]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
probs = tensor([[1.5011e-01, 3.4180e-03, 1.3903e-04, 9.7094

probs = tensor([[1.3349e-04, 2.8045e-03, 6.3123e-04, 5.5103e-05, 7.9627e-05, 1.6664e-03,
         2.4458e-03, 2.8453e-05, 1.3318e-03, 4.2120e-05, 7.4105e-04, 4.9666e-04,
         6.6809e-04, 8.3907e-04, 2.6186e-04, 5.8162e-05, 7.1066e-06, 1.3956e-04,
         9.9159e-05, 3.4030e-05, 7.4607e-04, 1.2297e-04, 8.2959e-05, 1.1879e-04,
         1.9137e-04, 1.3147e-04, 5.2862e-05, 1.8014e-04, 6.5525e-05, 2.8083e-05,
         1.4562e-04, 9.3529e-05, 1.5597e-05, 1.0004e-05, 9.0461e-05, 2.7052e-05,
         6.1847e-05, 9.0623e-05, 5.7973e-05, 8.9612e-02, 1.5357e-03, 1.0624e-02,
         3.3676e-04, 2.1211e-01, 2.0733e-03, 1.1934e-03, 2.0194e-01, 5.0381e-02,
         3.7954e-04, 6.0827e-02, 2.3332e-02, 1.0733e-03, 1.4246e-03, 2.0939e-01,
         1.7382e-03, 7.0896e-04, 3.7382e-02, 2.1652e-03, 4.2205e-02, 2.4739e-02,
         3.8828e-04, 4.1645e-04, 1.7473e-04, 8.8030e-03, 1.7150e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
probs = tensor([[1.5320e-02, 2.8630e-01, 6.2091e-03, 6.6871

# The mathematical trick in self-attention

In [15]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev,0)
        
print(x[0])
print()
print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [16]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = torch.softmax(wei, dim=1)
print(f"wei = \n{wei}")
(wei @ x)[0]

wei = 
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [17]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x

In [18]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [19]:
array1 = torch.randint(0, 10, (4,8,2))
array2 = torch.randint(0, 10, (8,2))

print(array1[0])
print(array2)

(array1 + array2)[0]

tensor([[0, 4],
        [0, 3],
        [8, 4],
        [0, 4],
        [1, 2],
        [5, 5],
        [7, 6],
        [9, 6]])
tensor([[7, 9],
        [4, 3],
        [8, 9],
        [3, 7],
        [8, 1],
        [4, 1],
        [6, 3],
        [2, 0]])


tensor([[ 7, 13],
        [ 4,  6],
        [16, 13],
        [ 3, 11],
        [ 9,  3],
        [ 9,  6],
        [13,  9],
        [11,  6]])